# Context of the problem

Cintas is a company that delivers clean uniforms to customers around the city.  While GPS can help navigate routes in real-time, route planning is done in advance based on sequencing stops based on geographical considerations. In fact, after speaking with a representative, I found that they plan routes entirely on zipcodes. The zipcode of a series of stops is not always the greatest predictor of what will complete the series of stops in the least amount of time. Can we predict traffic issues before they occur so that we may avoid them?

We will explore the Real-Time Traffic Incident Reports from Austin's Open Data Portal.
(https://data.austintexas.gov/Transportation-and-Mobility/Real-Time-Traffic-Incident-Reports/dx9v-zd7x)

# Import necessary libraries

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime

# Data Collection/Organization

I'm importing the spreadsheet changing the published/status date columns to datetime objects.

In [23]:
file = 'Real-Time_Traffic_Incident_Reports.csv'

In [24]:

df = pd.read_csv(file, parse_dates=['Published Date', 'Status Date'])

df.head()


,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status,Status Date
0,C163BCD1CF90C984E9EDA4DBA311BCA369A7D1A1_15288...,2018-06-13 06:35:59+00:00,Crash Service,"(30.283797,-97.741906)",30.283797,-97.741906,W 21ST ST & GUADALUPE ST,ARCHIVED,2018-06-13 09:00:03+00:00
1,6B20382196FB454E9FD06A33E60142902A2F0706_15288...,2018-06-13 10:15:36+00:00,Traffic Hazard,"(30.339593,-97.700963)",30.339593,-97.700963,400-717 E ANDERSON LN EB,ARCHIVED,2018-06-13 11:20:03+00:00
2,2A7A84FC69D56A2C379C274D2F0831442D3E7B8E_15871...,2020-04-17 21:25:03+00:00,Crash Urgent,"(30.329455,-97.638105)",30.329455,-97.638105,E US 290 HWY SVRD EB & EASTERN HEIGHTS BLVD,ARCHIVED,2020-04-17 21:55:03+00:00
3,89D3B75BB5932F10D25B5852A9902DA26C3DF11A_15871...,2020-04-17 21:40:52+00:00,Traffic Hazard,"(30.202806,-97.760701)",30.202806,-97.760701,5300-blk S Ih 35 Sb,ARCHIVED,2020-04-17 21:55:03+00:00
4,E7F9EFF0E81057A5C8FE41CABE4D9C6EEA82DAA0_15871...,2020-04-17 21:00:55+00:00,Crash Urgent,"(30.184265,-97.687339)",30.184265,-97.687339,4500-4720 S Us 183 Hwy Sb,ARCHIVED,2020-04-17 23:30:04+00:00


In [25]:
df.dtypes

Traffic Report ID                 object
Published Date       datetime64[ns, UTC]
Issue Reported                    object
Location                          object
Latitude                          object
Longitude                        float64
Address                           object
Status                            object
Status Date          datetime64[ns, UTC]
dtype: object

In [26]:
df.shape

(249650, 9)

There are 249,650 rows and 9 columns

# Data Wrangling

The column names have spaces in them. This will prove to be a problem with creating visualizations later so I'm going to change the variable names for the applicable columns.

In [27]:
df=df.rename(columns = {'Published Date':'published_date'})
df=df.rename(columns = {'Status Date':'status_date'})
df=df.rename(columns = {'Issue Reported':'issue_reported'})
df=df.rename(columns = {'Traffic Report ID':'tr_ID'})
df.head()

,tr_ID,published_date,issue_reported,Location,Latitude,Longitude,Address,Status,status_date
0,C163BCD1CF90C984E9EDA4DBA311BCA369A7D1A1_15288...,2018-06-13 06:35:59+00:00,Crash Service,"(30.283797,-97.741906)",30.283797,-97.741906,W 21ST ST & GUADALUPE ST,ARCHIVED,2018-06-13 09:00:03+00:00
1,6B20382196FB454E9FD06A33E60142902A2F0706_15288...,2018-06-13 10:15:36+00:00,Traffic Hazard,"(30.339593,-97.700963)",30.339593,-97.700963,400-717 E ANDERSON LN EB,ARCHIVED,2018-06-13 11:20:03+00:00
2,2A7A84FC69D56A2C379C274D2F0831442D3E7B8E_15871...,2020-04-17 21:25:03+00:00,Crash Urgent,"(30.329455,-97.638105)",30.329455,-97.638105,E US 290 HWY SVRD EB & EASTERN HEIGHTS BLVD,ARCHIVED,2020-04-17 21:55:03+00:00
3,89D3B75BB5932F10D25B5852A9902DA26C3DF11A_15871...,2020-04-17 21:40:52+00:00,Traffic Hazard,"(30.202806,-97.760701)",30.202806,-97.760701,5300-blk S Ih 35 Sb,ARCHIVED,2020-04-17 21:55:03+00:00
4,E7F9EFF0E81057A5C8FE41CABE4D9C6EEA82DAA0_15871...,2020-04-17 21:00:55+00:00,Crash Urgent,"(30.184265,-97.687339)",30.184265,-97.687339,4500-4720 S Us 183 Hwy Sb,ARCHIVED,2020-04-17 23:30:04+00:00


In [29]:
df['published_date'].min()

Timestamp('2017-09-26 16:11:00+0000', tz='UTC')

In [30]:
df['published_date'].max()

Timestamp('2021-12-13 19:35:58+0000', tz='UTC')

# Exploring the Data and Converting Datatypes

In [7]:
df.describe()
#Only one column appear as it's the only one with a numerical data type(float)

,Longitude
count,249571.000000
mean,-97.551690
std,4.211096
min,-98.816154
25%,-97.776950
50%,-97.734295
75%,-97.687038
max,0.000000


Having the 'Latitude' column remain an object makes no sense. It need to be converted into a float so that it may be used in conjunction with the 'Longitude' column.

In [8]:
df['Latitude'] = pd.to_numeric(df['Latitude'], errors = 'coerce') #Converting Latitude to numeric(float)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249650 entries, 0 to 249649
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   tr_ID           249650 non-null  object             
 1   published_date  249650 non-null  datetime64[ns, UTC]
 2   issue_reported  249650 non-null  object             
 3   Location        249107 non-null  object             
 4   Latitude        249525 non-null  float64            
 5   Longitude       249571 non-null  float64            
 6   Address         249650 non-null  object             
 7   Status          247957 non-null  object             
 8   status_date     249650 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), float64(2), object(5)
memory usage: 17.1+ MB


## Finding the Null Values

Finding the total number of nulls along with the total percentage of nulls per column.

In [9]:
missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
tr_ID,0,0.000000
published_date,0,0.000000
issue_reported,0,0.000000
Address,0,0.000000
status_date,0,0.000000
Longitude,79,0.031644
Latitude,125,0.050070
Location,543,0.217505
Status,1693,0.678149


## Null Summary 

The columns are missing some values are:
Longitude - 3%
Latitude - 5%
Location - 21%, This will be dropped later as it is redundant and has more null values
Status - 67%

### Drop the Status Column

The Status column will not be used for this analysis, it only inidicates whether a incident is archived or not. Everything in this dataset is archived.


In [10]:

df.drop(['Status'], axis=1, inplace=True)
df.head(3)

,tr_ID,published_date,issue_reported,Location,Latitude,Longitude,Address,status_date
0,C163BCD1CF90C984E9EDA4DBA311BCA369A7D1A1_15288...,2018-06-13 06:35:59+00:00,Crash Service,"(30.283797,-97.741906)",30.283797,-97.741906,W 21ST ST & GUADALUPE ST,2018-06-13 09:00:03+00:00
1,6B20382196FB454E9FD06A33E60142902A2F0706_15288...,2018-06-13 10:15:36+00:00,Traffic Hazard,"(30.339593,-97.700963)",30.339593,-97.700963,400-717 E ANDERSON LN EB,2018-06-13 11:20:03+00:00
2,2A7A84FC69D56A2C379C274D2F0831442D3E7B8E_15871...,2020-04-17 21:25:03+00:00,Crash Urgent,"(30.329455,-97.638105)",30.329455,-97.638105,E US 290 HWY SVRD EB & EASTERN HEIGHTS BLVD,2020-04-17 21:55:03+00:00


### Addressing Nulls for 'Latitude' and 'Longitude' Columns

In [11]:
df.dropna(subset=['Latitude'],inplace=True)
df.dropna(subset=['Longitude'],inplace=True)
#Dropping null values in the above columns

In [12]:
df.describe()

,Latitude,Longitude
count,249525.000000,249525.000000
mean,30.245476,-97.551618
std,1.308834,4.211477
min,0.000000,-98.816154
25%,30.231876,-97.776912
50%,30.294128,-97.734295
75%,30.373668,-97.687038
max,31.077333,0.000000


### Addressing 'Latitude' and 'Longitude' Rows with 0.0 Values

Clearly we have more than NaN values to worry about as 0.0 is essentially a NaN.

In [13]:
print(df.Latitude.min())
print(df.Longitude.max())

0.0
0.0


In [14]:
df = df[df.Latitude != 0]
df = df[df.Longitude != 0]

In [15]:
print(df.Latitude.min())
print(df.Longitude.max())

30.003883
-97.108986


#### What is the shape now?
We have now removed all NaN and 0 values from the Latitude and Longitude columns.

In [16]:
df.shape

(249061, 8)

### Recheck the null values

In [17]:
missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
tr_ID,0,0.0
published_date,0,0.0
issue_reported,0,0.0
Location,0,0.0
Latitude,0,0.0
Longitude,0,0.0
Address,0,0.0
status_date,0,0.0


### Dropping Traffic Report ID

In [18]:
duplicate = df[df.duplicated('tr_ID')]
duplicate

,tr_ID,published_date,issue_reported,Location,Latitude,Longitude,Address,status_date


This column has no pattern or purpose for prediction.

In [19]:
df.drop(['tr_ID'], axis=1, inplace=True)

In [20]:
df.head()

,published_date,issue_reported,Location,Latitude,Longitude,Address,status_date
0,2018-06-13 06:35:59+00:00,Crash Service,"(30.283797,-97.741906)",30.283797,-97.741906,W 21ST ST & GUADALUPE ST,2018-06-13 09:00:03+00:00
1,2018-06-13 10:15:36+00:00,Traffic Hazard,"(30.339593,-97.700963)",30.339593,-97.700963,400-717 E ANDERSON LN EB,2018-06-13 11:20:03+00:00
2,2020-04-17 21:25:03+00:00,Crash Urgent,"(30.329455,-97.638105)",30.329455,-97.638105,E US 290 HWY SVRD EB & EASTERN HEIGHTS BLVD,2020-04-17 21:55:03+00:00
3,2020-04-17 21:40:52+00:00,Traffic Hazard,"(30.202806,-97.760701)",30.202806,-97.760701,5300-blk S Ih 35 Sb,2020-04-17 21:55:03+00:00
4,2020-04-17 21:00:55+00:00,Crash Urgent,"(30.184265,-97.687339)",30.184265,-97.687339,4500-4720 S Us 183 Hwy Sb,2020-04-17 23:30:04+00:00


In [21]:
df.to_csv('traffic_data_cleaned.csv') 